<a href="https://colab.research.google.com/github/TristanFaine/Master_2_MLVC_Recognize_Handwritten_Equation/blob/main/code_template_explanation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task

We were given InkML files, which contain metadata and a list of strokes:  
[(0,0),(1;0)]...  
These are on-line handwritten mathematical expressions, we'll try to recognize them by using a LG(Labelled Graph) as intermediate output.  
The sequence of actions should look like this:  
1) Get each stroke  
2) Convert each stroke to symbols (can be optimized later)  
Two strategies available: Consider only consecutive strokes, or allow X(1) number of "time jumps" for each possible symbol.  
2.5) Make a classifier to ignore "junk" stroke combinations, such as a symbol created with every single stroke at the same time.  
3) Classify each constructed symbol with a neural network.

We'll handle spatial relations later  
We'll also handle the final decision later, but we can add a grammar or language model at that point, 

# Environment setup

Please import each file from the git repository to see how they work.  
TODO: link from the git repo, should be feasible.

Hiding my token for obvious reasons.

In [2]:
!git clone https://hiddentoken@github.com/TristanFaine/Master_2_MLVC_Recognize_Handwritten_Equation.git

Cloning into 'Master_2_MLVC_Recognize_Handwritten_Equation'...
remote: Enumerating objects: 119792, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 119792 (delta 34), reused 61 (delta 19), pack-reused 119711
Receiving objects: 100% (119792/119792), 46.21 MiB | 21.96 MiB/s, done.
Resolving deltas: 100% (38/38), done.
Checking out files: 100% (209488/209488), done.


In [3]:
%cd Master_2_MLVC_Recognize_Handwritten_Equation/code

/content/Master_2_MLVC_Recognize_Handwritten_Equation/code


We will first try out each script then explain what they're supposed to do and how to interpret their output.

## Data to train our classifiers

[This](https://uncloud.univ-nantes.fr/index.php/s/OdAtErZgxKGjsNy) contains a bunch of symbols (in datasymbol_iso/) and expressions (in FullExpressions/) to help train our future classifiers.



## Showcasing parts of code

## segmenter.py explanation

In [4]:
!python3 segmenter.py -i ../data/formulaire001-equation001.inkml -o ../data/bidule.lg

In [5]:
!python3 segmenter.py -i ../data/formulaire001-equation001.inkml

O,hyp0,*,1.0,0
O,hyp1,*,1.0,1
O,hyp2,*,1.0,2
O,hyp3,*,1.0,3
O,hyp4,*,1.0,4
O,hyp5,*,1.0,0,1
O,hyp6,*,1.0,1,2
O,hyp7,*,1.0,2,3
O,hyp8,*,1.0,3,4
O,hyp9,*,1.0,0,1,2
O,hyp10,*,1.0,1,2,3
O,hyp11,*,1.0,2,3,4
O,hyp12,*,1.0,0,1,2,3
O,hyp13,*,1.0,1,2,3,4



Each line of this output corresponds to a hypothesis, indicating the symbol type, the symbol with index (dummy for now_, then the symbol without index, then the supposed confidence of the model. We also have the strokes used next to these informations.

For now, we simply list every possible symbol, using every possible stroke in sequence, so if we have 4 strokes, we can make 13 possible symbols.  

segmentSelect.py can be used to remove some hypotheses that don't make sense, for instance, the symbol that could be constructed with stroke [0,1,2,3,4] is highly unlikely to appear, so after training our classifier, we'd remove it.  
Ground truth can be acquired from the lg files of the texts, since we have the strokes used as the 5th...i values.

## Use this subsection when testing stuff for now

In [31]:
#!git stash
#!git stash drop

Saved working directory and index state WIP on main: f67360a7f Updated path to git
Dropped refs/stash@{0} (df132c49108d8fbb0e13f85fa54fb2058dbcb68a)


In [47]:
#!git reset --soft HEAD^ 

In [48]:
#!git add .

In [49]:
#!git commit -m "Added modified version of AlexNet to available models"

[main 23649ee38] Added modified version of AlexNet to available models
 5 files changed, 53 insertions(+), 3 deletions(-)
 create mode 100644 code/output.png
 create mode 100644 code/resultMLP.png
 delete mode 100644 code/segmentSelector.nn


In [50]:
#!git push

Counting objects: 7, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (7/7), done.
Writing objects: 100% (7/7), 35.49 KiB | 17.75 MiB/s, done.
Total 7 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/TristanFaine/Master_2_MLVC_Recognize_Handwritten_Equation.git
   d1e450660..23649ee38  main -> main


In [32]:
#!git pull

Updating f67360a7f..d1e450660
Fast-forward
 code/CROHME_train_segmentSelector.py |   8 +++++---
 code/segmentSelector.nn              | Bin 412319 -> 249735 bytes
 2 files changed, 5 insertions(+), 3 deletions(-)


In [42]:
!python3 CROHME_train_segmentSelector.py

cuda:0
20000
('invalid', 'valid')
nb classes 2 , training size 12000, val size 4000, test size 4000
invalid invalid invalid invalid
AlexNet(
  (layer1): Sequential(
    (0): Conv2d(1, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer2): Sequential(
    (0): Conv2d(96, 384, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer3): Sequential(
    (0): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer4): Sequential(
    (0): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer5): Sequential(
    (0): C

## segmentSelect.py explanation

In [9]:
!python3 segmentSelect.py -o ../data/bidule2.lg ../data/formulaire001-equation001.inkml ../data/bidule.lg 

In [7]:
!python3 segmentSelect.py ../data/formulaire001-equation001.inkml ../data/bidule.lg 

Traceback (most recent call last):
  File "segmentSelect.py", line 24, in <module>
    model.load_state_dict(dict(torch.load('segmentSelector.nn')))
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 1605, in load_state_dict
    self.__class__.__name__, "\n\t".join(error_msgs)))
RuntimeError: Error(s) in loading state_dict for SegmentSelector:
	Missing key(s) in state_dict: "conv1.weight", "conv1.bias", "conv2.weight", "conv2.bias", "fc3.weight", "fc3.bias". 
	size mismatch for fc1.weight: copying a param with shape torch.Size([100, 1024]) from checkpoint, the shape in current model is torch.Size([120, 400]).
	size mismatch for fc1.bias: copying a param with shape torch.Size([100]) from checkpoint, the shape in current model is torch.Size([120]).
	size mismatch for fc2.weight: copying a param with shape torch.Size([2, 100]) from checkpoint, the shape in current model is torch.Size([84, 120]).
	size mismatch for fc2.bias: copying a param with shape torch.Si

This script takes as input the initial inkml file, alongside the "prototype" lg file: We combine the stroke combinations alongside the inkml data to generate images, then we check whether these make sense as a symbol, no matter the context.

For now this simply gives a fully random probability that the images are valid symbols.

Need to change two things in there, first adding a classifier to check the validity of supposed symbols,  
then change the threshold based on empirical evidence or what I feel like at the time.

##segmentReco.py explanation

In [ ]:
!chmod 777 ./symbolReco.py

In [ ]:
!python3 symbolReco.py ../data/formulaire001-equation001.inkml ../data/bidule2.lg

O,hyp0,r,0.059779757207917304,0
O,hyp0,geq,0.05030687013768514,0
O,hyp1,2,0.05648836776140208,1
O,hyp1,6,0.056168315175164935,1
O,hyp1,8,0.05960923199570466,1
O,hyp1,=,0.05475807870783242,1
O,hyp1,geq,0.056816822917386364,1
O,hyp3,f,0.0510902919404698,3
O,hyp3,n,0.0586707731068381,3
O,hyp3,u,0.053721497506933835,3
O,hyp3,3,0.055092772863820916,3
O,hyp4,b,0.05192545736402091,4
O,hyp4,leq,0.07015718890027181,4
O,hyp5,s,0.06121838413470821,0,1
O,hyp5,A,0.050391613253440264,0,1
O,hyp5,X,0.072619141223795,0,1
O,hyp6,(,0.05046390880442578,1,2
O,hyp7,p,0.06723378695795225,2,3
O,hyp7,y,0.05172590875612772,2,3
O,hyp7,1,0.05240988369270244,2,3
O,hyp7,geq,0.058408507485593536,2,3
O,hyp8,Z,0.053352030138174675,3,4
O,hyp9,h,0.051727432180693006,0,1,2
O,hyp9,C,0.05847255666598241,0,1,2
O,hyp9,(,0.05560318869409104,0,1,2
O,hyp9,geq,0.05338792763291579,0,1,2
O,hyp10,r,0.055058494864859676,1,2,3
O,hyp10,4,0.050618596960434595,1,2,3
O,hyp11,c,0.05128946473017116,2,3,4
O,hyp11,j,0.06079852247795053,2,3,4